In [17]:
!pip install pymc3
!pip install arviz

In [18]:
import arviz as az
import pymc3 as pm
import pandas as pd

In [19]:
from google.colab import files

In [20]:
result = files.upload()

In [9]:
whos

Variable   Type      Data/Info
------------------------------
az         module    <module 'arviz' from '/us<...>kages/arviz/__init__.py'>
files      module    <module 'google.colab.fil<...>s/google/colab/files.py'>
pm         module    <module 'pymc3' from '/us<...>kages/pymc3/__init__.py'>
result     dict      n=1


In [10]:
result

{'FOREST.XLS': b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00>\x00\x03\x00\xfe\xff\t\x00\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x1f\x00\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\xfe\xff\xff\xff\x00\x00\x00\x00\xfe\xff\xff\xff\x00\x00\x00\x00\x1e\x00\x00\x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff

In [21]:
df = pd.read_excel("HPRICE.XLS")

In [22]:
y = df['sale price']
x1 = df['lot size']
x2 = df['#bedroom']

In [25]:
niter = 2000

with pm.Model() as model:
  alpha = pm.Normal('alpha', mu=0, sigma=10)
  beta1 = pm.Normal('beta1', mu=0, sigma=10)
  beta2 = pm.Normal('beta2', mu=0, sigma=10)
  eps = pm.HalfCauchy('eps', beta=5)

  mu = alpha + beta1 * x1 + beta2 * x2
  y_pred = pm.Normal('y_pred', mu=mu, observed=y)
  #trace = pm.sample(500)
  start = pm.find_MAP() # Use MAP estimate (optimization) as the initial state for MCMC
  step = pm.Metropolis() # Have a choice of samplers
  trace = pm.sample(niter, step, start, random_seed=123, progressbar=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  
Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [eps]
>Metropolis: [beta2]
>Metropolis: [beta1]
>Metropolis: [alpha]


/usr/local/lib/python3.7/dist-packages/pymc3/step_methods/metropolis.py:226: RuntimeWarning: overflow encountered in exp
  "accept": np.exp(accept),


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 4 seconds.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


In [28]:
pm.summary(trace, kind='stats')

Got error No model on context stack. trying to find log_likelihood in translation.
/usr/local/lib/python3.7/dist-packages/arviz/data/io_pymc3_3x.py:102: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


,mean,sd,hdi_3%,hdi_97%
alpha,975.759,344.292,405.671,1616.339
beta1,11.593,0.231,11.159,11.958
beta2,971.635,345.982,403.299,1614.790
eps,13.414,61.880,0.004,33.331
